In [1]:
import streamlit as st
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from docx import Document
import PyPDF2
import os
import tempfile
import tiktoken

In [12]:
pip install pycryptodome

Note: you may need to restart the kernel to use updated packages.


In [4]:
ls

CD_seller_signed.pdf                    app_scratch_book.ipynb
Hatching_a_story.docx                   final_signed_offer.pdf
LICENSE                                 image2.png
Purchase_and_sale_final.pdf             image3.png
README.md                               image5.png
RSM_packing_list.docx                   image6.png
RSM_packing_list_2.docx                 img.png
Software_Engineering_Practices_TOP.txt  requirements.txt
app.py                                  temp/
app1.py                                 ~$tching_a_story.docx


In [5]:
pip show pycryptodome

Name: pycryptodome
Version: 3.20.0
Summary: Cryptographic library for Python
Home-page: https://www.pycryptodome.org
Author: Helder Eijs
Author-email: helderijs@gmail.com
License: BSD, Public Domain
Location: /Users/alexanderarefolov/Dropbox/Coding_Projects/knowledge_retrieval_LLM_chatbot_Streamlit_app/.venv/lib/python3.12/site-packages
Requires: 
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [3]:
load_dotenv()

True

In [4]:
# helper function to process the input text file, remove empty lines and unneeded formatting marks
def process_input_file(input_file_path):
    '''
    process_input_text() helper function takes the input file in txt, docx or pdf format
    as an argument and removes empty lines and non-essential characters. The output is saved
    in a temporary directory.
    
    Parameters:
        input_file_path (str): path to the input text file
    
    Returns:
        processed temporary text file path saved in temp/
    '''
    # Create a temporary file in the same directory as the input file
    temp_dir = os.path.join(os.path.dirname(input_file_path), "temp")
    os.makedirs(temp_dir, exist_ok = True)

    temp_file = tempfile.NamedTemporaryFile(mode = 'w', delete = False, dir = temp_dir, encoding = 'UTF-8')

    try:
        file_extension = os.path.splitext(input_file_path)[1].lower()

        # Read the contents of the file based on its type
        if file_extension == '.txt':
            with open(input_file_path, 'r', encoding='UTF-8') as input_file:
                lines = input_file.readlines()
        elif file_extension == '.docx':
            doc = Document(input_file_path)
            lines = [p.text for p in doc.paragraphs]
        elif file_extension == '.pdf':
            with open(input_file_path, 'rb') as input_file:
                reader = PyPDF2.PdfReader(input_file)
                lines = []
                for page_num in range(len(reader.pages)):
                    page = reader.pages[page_num] 
                    lines.append(page.extract_text())
        else:
            raise ValueError("Unsupported file format: " + file_extension)

        # Remove empty lines and lines consisting only of '-' or '_'
        non_empty_lines = [line.strip() for line in lines if line.strip() and not all(char in {'-', '_'} for char in line.strip())]

        # Write processed text to the temporary file
        temp_file.write('\n'.join(non_empty_lines))
    finally:
        # Close the temporary file
        temp_file.close()

    # Get the path of the temporary file
    temp_file_path = temp_file.name

    return temp_file_path

In [12]:
input_file_path = "Purchase_and_sale_final.pdf"

In [8]:
input_file_path = "final_signed_offer.pdf"

In [17]:
input_file_path = "CD_seller_signed.pdf"

In [18]:
with open(input_file_path, 'rb') as input_file:
    reader = PyPDF2.PdfReader(input_file)
    lines = []
    for page_num in range(len(reader.pages)):
        page = reader.pages[page_num] 
        lines.append(page.extract_text())

lines

['', '', '']

In [19]:
len(lines)

3

In [20]:
lines[0]

''

In [21]:
len(lines[0])

0